まずはデータセットを取得しよう。

In [1]:
# 古典のデータセットをscikit-learnから取得
from sklearn import datasets
from sklearn.model_selection import train_test_split

X, y = datasets.fetch_openml('mnist_784', version=1, return_X_y=True)
X = X[:60000]
y = y[:60000]
classic_X_train, classic_X_test, classic_y_train, classic_y_test = train_test_split(X / 255, # ピクセル値が 0 - 1 になるようにする
                                                        y.astype('int64'), # 正解データを数値にする
                                                        stratify = y,
                                                        random_state=0)

/workspace/qulacs_dataset/.venv/lib/python3.9/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
# 量子のデータセットをqulacs_datasetから取得
from qulacs_dataset.mnist import load_large_f90
from qulacs import QuantumState, QuantumCircuit
import numpy as np

def generate_data(circuit:QuantumCircuit) -> np.array:
  state = QuantumState(10)
  circuit.update_quantum_state(state)
  return state.get_vector().real

items = load_large_f90()
quantum_X_train = [generate_data(x) for x in items["circuit"][:450]]
quantum_y_train = items["label"][:450]
quantum_X_test = [generate_data(x) for x in items["circuit"][450:]]
quantum_y_test = items["label"][450:]

100%|██████████| 60000/60000 [00:06<00:00, 9012.95it/s]


# ロジスティック回帰

まずは定番のロジスティック回帰から実験してみる。

In [2]:
from sklearn.linear_model import LogisticRegression, LinearRegression

clf = LogisticRegression(solver='lbfgs', multi_class='auto')
clf.fit(classic_X_train, classic_y_train) # 学習
print('古典情報を入力としたときのaccuracy_score: %.3f' % clf.score(classic_X_test, classic_y_test)) # 検証

NameError: name 'classic_X_train' is not defined

In [3]:
clf = LogisticRegression(solver='lbfgs', multi_class='auto')
clf.fit(quantum_X_train, quantum_y_train) # 学習
print('量子状態を入力としたときのaccuracy_score: %.3f' % clf.score(quantum_X_test, quantum_y_test)) # 検証

量子状態を入力としたときのaccuracy_score: 0.753


## SVM


In [4]:
from sklearn import svm

model = svm.SVC()
model.fit(classic_X_train, classic_y_train) # 学習
print('古典情報を入力としたときのaccuracy_score: %.3f' % model.score(classic_X_test, classic_y_test)) # 検証

NameError: name 'classic_X_train' is not defined

In [6]:
model = svm.SVC()
model.fit(quantum_X_train, quantum_y_train) # 学習
print('古典情報を入力としたときのaccuracy_score: %.3f' % model.score(quantum_X_test, quantum_y_test)) # 検証

古典情報を入力としたときのaccuracy_score: 0.971
